In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
stocks=['MSFT','AAPL','NVDA','GOOGL','AMZN','META','BRK-B','LLY','AVGO','V','JPM','TSLA','WMT','XOM','UNH','MA','PG','JNJ','HD','ORCL','MRK','COST','ABBV','CVX','BAC','CRM','NFLX','KO','AMD','PEP','ADBE','TMO','DIS','WFC','MCD','CSCO','TMUS','ABT','QCOM','CAT','DHR','INTU','GE','IBM','VZ','AMAT','AXP','CMCSA','NOW','COP','INTC','TXN','UBER','BX','MS','PFE','NKE','AMGN','PM','UNP','RTX','ISRG','SPGI','GS','LOW','NEE','MU','SCHW','SYK','HON','PGR','UPS','LRCX','ELV','BKNG','T','BLK','C','DE','LMT','TJX','BA','ABNB','VRTX','BSX','ADP','PLD','CI','SBUX','REGN','MMC','BMY','ADI','PANW','MDLZ','KLAC','SCCO','FI','CVS','DELL','KKR','GILD','WM','HCA','ANET','SNPS','AMT','CMG','CDNS','SHW','GD','EOG','SO','TGT','CME','ITW','ICE','MPC','DUK','MO','SLB','FCX','CL','CRWD','ZTS','EQIX','PH','MCK','MAR','MCO','TDG','CTAS','WDAY','PSX','BDX','APH','NOC','CSX','PYPL','FDX','ORLY','EMR','ECL','PXD','USB','EPD','APO','PCAR','RSG','PNC','OXY','CEG','MRVL','MSI','MNST','ROP','SMCI','VLO','NSC','DASH','EW','COF','CPRT','COIN','DXCM','ET','WELL','APD','AZO','HLT','MMM','AJG','MET','SNOW','EL','AIG','FTNT','GM','CARR','DHI','COR','TFC','TRV','STZ','F','GWW','NUE','HES','AFL','PSA','IBKR','ADSK','MCHP','SPG','WMB','ODFL','OKE','SQ','PLTR']
stocks=[stock.replace('-','.') for stock in stocks][:100]

In [3]:
from order_routerv2 import *
import tensorflow as tf
from keras.models import load_model
from utils import*
from concurrent.futures import ThreadPoolExecutor,ProcessPoolExecutor
from datetime import*
import pickle as pkl


from alpaca.data.historical import *
from alpaca.data.requests import *
from alpaca.data.timeframe import *
from alpaca.data.enums import *
from alpaca.trading.client import *
from alpaca.trading.requests import * 
from alpaca.trading.enums import *

with open('multi_scaler3.pkl','rb') as f:
    multi_scaler=pkl.load(f)
with open('encoder.pkl','rb') as f:
    encoder=pkl.load(f)
    


In [153]:
trader=Trader(model=load_model('model3.keras'),multi_scaler=multi_scaler,encoder=encoder)


In [162]:
def rd(dt):
    # Extract minutes, seconds, and microseconds
    minute = dt.minute
    seconds = dt.second
    microseconds = dt.microsecond
    
    # Find how many minutes to subtract to round down to the nearest 15-minute interval
    minute_to_subtract = minute % 15
    
    # Subtract the extra minutes, all seconds, and all microseconds
    rounded_dt = dt - timedelta(minutes=minute_to_subtract, seconds=seconds, microseconds=microseconds)

In [1]:
from time import sleep
from IPython.display import clear_output
from datetime import *
def rd(dt,t):
    return datetime.fromtimestamp(np.floor(dt.timestamp()/(t*60))*(t*60),timezone.utc)

while True:
    dt=datetime.now(timezone.utc)
    if dt.minute in [00,15,30,45]:
                
        positions={i.symbol:i for i in trader.trading_client.get_all_positions()}
        bars=trader.get_bars(stocks,rd(datetime.now(timezone.utc)))
        arrs=trader.preprocess_bars_v2(bars,stocks)
        predictions=trader.predict(arrs)
        trades=dict(zip(stocks,predictions))
        prices=bars.groupby('symbol').last()['close'].to_dict()
        orders=trader.generate_orders(trades,prices,positions)
        trader.trading_client.cancel_orders()
        trader.execute(orders[:])
        print('done')
        sleep(60)
    else:
        clear_output(wait=True)
        print('sleeping ',dt)
        sleep(.1)

NameError: name 'datetime' is not defined

In [3]:
def rd(dt):
    dt=dt.timestamp()
    return dt
dt=datetime.now()
print(rd(dt))

1714411602.102352


In [4]:
from datetime import *

In [25]:
def rd(dt,t):
    return datetime.fromtimestamp(np.floor(dt.timestamp()/(t*60))*(t*60),timezone.utc)


datetime.datetime(2024, 4, 29, 17, 30, tzinfo=datetime.timezone.utc)

In [19]:
import numpy as np